In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def outliers(data, fator=1.5):
    data_limpa = data.copy()
    
    for col in data_limpa.select_dtypes(include=[np.number]).columns:
        Q1 = data_limpa[col].quantile(0.25)
        Q3 = data_limpa[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - (fator * IQR)
        upper_bound = Q3 + (fator * IQR)
        data_limpa[col] = np.clip(data_limpa[col], lower_bound, upper_bound)
    
    return data_limpa

In [3]:
data_treino = pd.read_csv('/home/caio/github/k-3/data/train.csv')
data_teste = pd.read_csv('/home/caio/github/k-3/data/test.csv')

In [4]:
data_treino_limpo = outliers(data_treino)
data_teste_limpo = outliers(data_teste)

In [5]:
feature_treino = data_treino_limpo.drop(columns=['rainfall']) # Feature Treino
target_treino = data_treino.pop('rainfall')                   # Target Treino
feature_final_imperfeito = data_teste_limpo
feature_final = feature_final_imperfeito.fillna(feature_final_imperfeito.mean())

In [6]:
X_train, X_val, y_train, y_val = train_test_split(feature_treino, target_treino, test_size=0.2, random_state=69)

model_meta = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer (regressão)
])

2025-03-10 01:40:06.013034: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [14]:


# Compilar o modelo
model_meta.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='mean_squared_error')


# Treinar o meta-modelo
model_meta.fit(X_train, y_train, epochs=40, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/40
55/55 [==============================] - 1s 10ms/step - loss: 0.1030 - val_loss: 0.1295
Epoch 2/40
55/55 [==============================] - 0s 8ms/step - loss: 0.1008 - val_loss: 0.1315
Epoch 3/40
55/55 [==============================] - 0s 8ms/step - loss: 0.1015 - val_loss: 0.1269
Epoch 4/40
55/55 [==============================] - 0s 9ms/step - loss: 0.1034 - val_loss: 0.1270
Epoch 5/40
55/55 [==============================] - 1s 9ms/step - loss: 0.1020 - val_loss: 0.1274
Epoch 6/40
55/55 [==============================] - 0s 6ms/step - loss: 0.1023 - val_loss: 0.1310
Epoch 7/40
55/55 [==============================] - 0s 7ms/step - loss: 0.1038 - val_loss: 0.1342
Epoch 8/40
55/55 [==============================] - 0s 9ms/step - loss: 0.1093 - val_loss: 0.1310
Epoch 9/40
55/55 [==============================] - 0s 8ms/step - loss: 0.1045 - val_loss: 0.1378
Epoch 10/40
55/55 [==============================] - 0s 5ms/step - loss: 0.1059 - val_loss: 0.1270
Epoch 11/40
55/55 

In [15]:
previsao = model_meta.predict(data_teste)
previsao1 = model_meta.predict(feature_treino)

69/69 [==============================] - 0s 3ms/step


In [13]:
caminho_destino = 'previsoes/linear/Neural_v1.csv'
resultado = pd.DataFrame({
    'id': data_teste['id'],
    'rainfall': previsao.flatten() 
})

resultado.to_csv(caminho_destino, index=False)

In [19]:
caminho_meta = '/home/caio/github/k-3/meta_features/neural_final.csv'
meta_f = pd.DataFrame({
    'neural': previsao.flatten() 
})

meta_f.to_csv(caminho_meta, index=False)

In [20]:
caminho_meta = '/home/caio/github/k-3/meta_features/neural_teste.csv'
meta_f = pd.DataFrame({
    'neural_treino': previsao1.flatten() 
})

meta_f.to_csv(caminho_meta, index=False)